## Process Movie

In [1]:
import matplotlib.pyplot as plt
import cv2, os, time, pickle, shutil, glob
import numpy as np
from lesson_functions import *
import pickle
import os
from car_finder import SingleFrameCarFinder
from car_finder import VehicleIdentifier
from moviepy.editor import VideoFileClip

svm_file = 'svm-yuv-16x2x11.p'

In [ ]:
# load parameters saved by the training process

with open(svm_file, 'rb') as f:
    data = pickle.load(f)
    svc = data['svm']
    X_scaler = data['scaler']
    params = data['params']

# load identifier and pass it the svc, scaler and params
identifier = VehicleIdentifier(params, svc, X_scaler)

video_filename = 'project_video.mp4'
clip = VideoFileClip(video_filename)

video_with_cars = clip.fl_image(identifier.process_frame)

output = os.path.splitext(video_filename)
output_file = output[0] + "_check" + output[1]

video_with_cars.write_videofile(output_file, audio=False)

print('File saved to:', output_file)

## Make Movie Out of Frames

In [1]:
from moviepy.editor import VideoFileClip
from moviepy.editor import ImageSequenceClip

files = './frame-cars/'
output = './frame-cars/0.mp4'

clip = ImageSequenceClip(files, fps=25)
clip.to_videofile(output, fps=25) # many options available


[MoviePy] >>>> Building video ./frame-cars/0.mp4
[MoviePy] Writing video ./frame-cars/0.mp4


100%|██████████████████████████████████████████████████████████████████████████████| 1260/1260 [00:29<00:00, 42.26it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./frame-cars/0.mp4 



## Get Frames Into Folder

In [ ]:
from moviepy.editor import VideoFileClip

video_filename = 'project_video-29-39.mp4'

output = os.path.splitext(video_filename)
output_folder = "{}-frames".format(output[0])

if os.path.exists(output_folder):
    print('removing folder', output_folder)
    shutil.rmtree(output_folder)

os.mkdir(output_folder)
print('Folder created', output_folder)

clip = VideoFileClip(video_filename)

for i, image in enumerate(clip.iter_frames()):
    mpimg.imsave("{}/{:04d}.jpg".format(output_folder, i), image, format="jpeg")
    print("{} generated".format(i), end='\r')

print("done")

## Subclip

In [ ]:
from moviepy.editor import VideoFileClip
clip = VideoFileClip("project_video.mp4").subclip(29, 39)
clip.write_videofile('project_video-29-39.mp4', audio=False)

## Generate features only

In order to speed up the process, all frames' features for those windows that might have a car are saved to a folder. CarFinder can then use those features rather than extracting them again

In [ ]:
frame_folder = 'project_video-frames'
output_folder = "{}/car_windows/".format(frame_folder)
if os.path.exists(output_folder):
    print('removing folder', output_folder)
    shutil.rmtree(output_folder)

os.mkdir(output_folder)
print('Folder created', output_folder)

with open(svm_file, 'rb') as f:
    data = pickle.load(f)
    svc = data['svm']
    X_scaler = data['scaler']
    params = data['params']
    
print('Data Loaded')
print('SVC:', svc)
print('Params', params)

files = glob.glob('./{}/*.jpg'.format(frame_folder))
for index, filename in enumerate(files):
    # read RGB since thats what video will give us and then our function
    # internally converts it to LAB
    img_cs = load_image(filename, params['color_space'])

    cf = SingleFrameCarFinder(svc, X_scaler, params)

    t1 = time.time()
    car_windows, window_features = cf.predict_cars(img_cs, ret_features = True)
    t2 = time.time()

    data_to_save = {'windows':car_windows, 'features':window_features}

    basename = os.path.basename(filename)
    basename = os.path.splitext(basename)[0]

    output_filename = './{}/{}.p'.format(output_folder, basename)
    with open(output_filename, 'wb') as f:
        pickle.dump(data_to_save, f)

    print('Frame {}, time taken {:.3f} secs. Saved to {}'.format(index, t2 - t1, output_filename))
